In [ ]:
%load_ext autoreload
%autoreload 2

from pymedphys_monomanage.tree import PackageTree
import networkx as nx
from copy import copy

In [ ]:
package_tree = PackageTree('../../packages')

In [ ]:
package_tree.package_dependencies_digraph

In [ ]:
package_tree.roots

In [ ]:
modules = list(package_tree.digraph.neighbors('pymedphys_analysis'))
modules

In [ ]:
internal_packages = copy(package_tree.roots)
internal_packages.remove('pymedphys')

In [ ]:
module_paths = [
    item
    for package in internal_packages
    for item in package_tree.digraph.neighbors(package)
]

modules = {
    item: os.path.splitext(item)[0].replace(os.sep, '.')
    for item in module_paths
}

modules

In [ ]:
module_digraph = nx.DiGraph()





In [ ]:
dependencies = {
    module.replace(os.sep, '.'): [
        '.'.join(item.split('.')[0:2])
        for item in
        package_tree.descendants_dependencies(module)['internal_module'] + package_tree.descendants_dependencies(module)['internal_package']
    ]
    for module in modules.keys()
}

dependencies

In [ ]:
dependents = {
    key: [] for key in dependencies.keys()
}
for key, values in dependencies.items():
    for item in values:
        dependents[item].append(key)
        
dependents

In [ ]:
current_modules = [
    item.replace(os.sep, '.')
    for item in package_tree.digraph.neighbors('pymedphys_analysis')
]
current_modules

In [ ]:
def remove_prefix(text, prefix):
    if text.startswith(prefix):
        return text[len(prefix):]
    else:
        return text


In [ ]:
graphed_module = 'pymedphys_monomanage'


current_modules = [
    item.replace(os.sep, '.')
    for item in package_tree.digraph.neighbors(graphed_module)
]
current_modules


def simplify(text):
    text = remove_prefix(text, "{}.".format(graphed_module))
    text = remove_prefix(text, 'pymedphys_')
    
    return text

In [ ]:
current_modules

In [ ]:
module_internal_relationships = {
    module.replace(os.sep, '.'): [
        '.'.join(item.split('.')[0:2])
        for item in
        package_tree.descendants_dependencies(module)['internal_module']
    ]
    for module in package_tree.digraph.neighbors(graphed_module)
}

module_internal_relationships

In [ ]:
dag = nx.DiGraph()

for key, values in module_internal_relationships.items():
    dag.add_node(key)
    dag.add_nodes_from(values)
    edge_tuples = [
        (key, value) for value in values
    ]
    dag.add_edges_from(edge_tuples)
    

dag.edges()


In [ ]:
def get_levels(dag):

    topological = list(nx.topological_sort(dag))

    level_map = {}
    for package in topological[::-1]:
        depencencies = nx.descendants(dag, package)
        levels = {0}
        for dependency in depencencies:
            try:
                levels.add(level_map[dependency])
            except KeyError:
                pass
        max_level = max(levels)
        level_map[package] = max_level + 1

    levels = {
        level: []
        for level in range(max(level_map.values()) + 1)
    }
    for package, level in level_map.items():
        levels[level].append(package)
    
    return levels
    
levels = get_levels(dag)
levels

In [ ]:
nodes = ""

for level in range(max(levels.keys()) + 1):
    if levels[level]:
        trimmed_nodes = [
            simplify(node) for node in levels[level]
        ]

        grouped_packages = '"; "'.join(trimmed_nodes)
        nodes += """
        {{ rank = same; "{}"; }}
        """.format(grouped_packages)
        
print(nodes)

In [ ]:
edges = ""
current_packages = ""

current_dependents = set()
current_dependencies = set()


for module in current_modules:
    module_repr = simplify(module)
    current_packages += '"{}";\n'.format(module_repr)
    
    for dependency in dependencies[module]:
        simplified = simplify(dependency)
        edges += '"{}" -> "{}";\n'.format(module_repr, simplified)
        if not dependency in current_modules:
            current_dependencies.add(simplified)
        
    for dependent in dependents[module]:
        simplified = simplify(dependent)
        edges += '"{}" -> "{}";\n'.format(simplified, module_repr)
        if not dependent in current_modules:
            current_dependents.add(simplified)
        

external_ranks = ""
if current_dependents:
    grouped_dependents = '"; "'.join(current_dependents)
    external_ranks += '{{ rank = same; "{}"; }}\n'.format(grouped_dependents)

if current_dependencies:
    grouped_dependencies = '"; "'.join(current_dependencies)
    external_ranks += '{{ rank = same; "{}"; }}\n'.format(grouped_dependencies)
        
print(edges)

In [ ]:
dot_file_contents = """
    strict digraph  {{
        rankdir = LR;
        
    
    {}
        
        subgraph cluster_0 {{
            {}
            label = "{}";
            style = dashed;
            {}
        }}
        
        {}
    }}
""".format(external_ranks, current_packages, graphed_module, nodes, edges)
print(dot_file_contents)

In [ ]:
	subgraph cluster_0 {
		style=filled;
		color=lightgrey;
		node [style=filled,color=white];
		a0 -> a1 -> a2 -> a3;
		label = "process #1";
	}

In [ ]:
package_tree.descendants_dependencies('pymedphys_monomanage/parse')

In [ ]:
package_tree.imports

In [ ]:
list(package_tree.digraph.nodes)